## Progress Log: To‑Do List Manager

**Last Updated:** July 2, 2025  
**Author:** Jerick C. Jualo

I’ve been steadily enhancing my To‑Do List Manager. Here’s what I’ve added, refactored, and polished so far:

---

### 1. Authentication & Account Management (`classes_functs.py`)

- **Persistent Account Registry**  
  - Introduced `Account.LIST_ACC_ID`, `Account.ALL_ACC`, `Account.NO_ACCOUNTS`, and `Account.DETAILS_ACCOUNT` to track every user I create.

- **Login Returns the Account Object**  
  - Updated `log_in()` so it now returns the actual `Account` instance (`logged_acc`) instead of just `True`.

- **Per‑Account Task Store**  
  - Switched from a one‑slot dict to a proper `self.tasks` dict keyed by each task’s `task_id`.  
  - Renamed the store from `list_tasks` to `tasks` for clarity.

- **Account‑Scoped Task Factory**  
  - Moved `create_task()` into the `Account` class. Now it:
  1. Prompts for task name, description, and status  
  2. Instantiates `Task(..., owner_id=self.acc_id)`  
  3. Inserts the new task into `self.tasks[new_task.task_id]`  
  4. Returns the `Task` object for further actions

- **Account Detail & Task Views**  
  - `view_tasks()` prints a neat list of all tasks for the current account (ID, name, status, date created).  
  - `full_acc_details()` shows account metadata plus a full dump of each `Task` via its `__repr__`.

---

### 2. Task Model & Behaviors (`tasks.py`)

- **One‑to‑Many Ownership**  
  - Added `self.owner_id` in `Task.__init__` so every task knows which `acc_id` created it.  
  - Maintained global registries (`Task.ALL_TASKS`, `Task.NO_TASKS`, `Task.LIST_TASK_ID`) for analytics or bulk exports.

- **Improved String Representations**  
  - Implemented both `__repr__` (for concise debugging) and `__str__` (for multi‑line, pretty‑printed display).

- **Editable Fields with Confirmation**  
  - Methods `rename()`, `change_description()`, and `change_status()` now:  
    1. Validate input (e.g., non‑empty name, valid status)  
    2. Prompt for “yes/no” confirmation  
    3. Apply the change or cancel gracefully

- **Due‑Date Setter**  
  - Interactive `set_due_date()` with month/day validation, returning a `datetime` object for serialization.

---

### 3. Main Flow & Menuing (`main_module.py` + `menu.py`)

- **Central Entry Point**  
  - `main_module.py` handles the welcome screen, login loop, then calls `super_menu(logged_acc)` once authenticated.

- **Interactive Super‑Menu**  
  - Options include:  
    - View account details  
    - View my tasks  
    - Create a new task  
    - Edit an existing task  
    - Log out

- **Edit Workflow**  
  - When choosing “Edit a Task”:  
    1. Reprint current tasks  
    2. Prompt for `task_id`  
    3. Call `edit_task(logged_acc, task_id)`, which dispatches to `task.rename()`, `task.change_description()`, or `task.change_status()` as needed

---

### 4. What’s Next

1. **Deleting & Archiving Tasks**  
   - Add `delete_task(task_id)` to remove tasks from both `self.tasks` and `Task.ALL_TASKS`.

2. **Editing Due Dates from the Menu**  
   - Expose `task.set_due_date()` in the menu for adding or changing deadlines interactively.

3. **Persistence Layer**  
   - Implement `serialize()` / `deserialize()` on both classes to read/write JSON or CSV at startup/shutdown.


In [ ]:
# FROM MAIN_MODULE

from classes_functs import log_in
from classes_functs import Account
from menu import super_menu

# Introduction

print("""welcome to my To-Do List Manager
      
Coded by: Jerick C. Jualo""")

print("\n=======================================================================================================================================================")
print("Welcome to the To-Do Task Manager!")
print("=======================================================================================================================================================")
print("This program will help you track your goals and weekly schedule for Python, SQL Learning, and Math Study.")
print("Also, it will help you create your tasks for today and check the status of your tasks.")
print("Please log in to continue.")
print("=======================================================================================================================================================\n")


# Ensure the user is logged in before proceeding 

logged_acc = None
# Loop until the user successfully logs in
while not logged_acc:
    logged_acc = log_in()

super_menu(logged_acc)


# After the user logs in, the super_menu function will be called to display the main menu and handle further interactions.



In [ ]:
#FROM classes_functs.py 

from datetime import datetime 
from tasks import Task

#CLASS FOR ACCOUNT
class Account:
    
    # Class Attributes
    LIST_ACC_ID: list[int] = []
    ALL_ACC: list[str] = []
    NO_ACCOUNTS: int = 0
    DETAILS_ACCOUNT = {}
        
    
    def __init__(self, username: str, password: str):

        # Username, Password, Account ID
        key = username.lower()
        self.username = username
        self.password = password
        
        next_id: int = Account.LIST_ACC_ID[-1] + 1 if Account.LIST_ACC_ID else 1

        self.acc_id: int = next_id # Owner ID on Task Class
        
        #Task related
        
        self.tasks = {}
        

        #Actions to be executed for Class Attributes
        Account.LIST_ACC_ID.append(next_id)
        Account.ALL_ACC.append(self)
        Account.NO_ACCOUNTS += 1
        Account.DETAILS_ACCOUNT[key] = password 
        


    # Instance Methods

    #Account Details Display Methods

    def display_NO_ACCOUNTS(self) -> str:
        return f"Total Number of Accounts: {Account.NO_ACCOUNTS}"
    
    
    def display_accounts(self) -> str:
        return f"Account Details: {Account.DETAILS_ACCOUNT}"

    def view_acc_id(self) -> str:
        return f"Account: {self.username} ID: {self.acc_id}"
    
    
    def full_acc_details(self) -> str:
        print(f"Account Details for {self.username}:")
        print(f"Username: {self.username}")
        print(f"Account ID: {self.acc_id}")
        print(f"Tasks: {self.tasks}")

        for x in self.tasks.values():
            print(x)
        print("==========================")
        return ""

        
    def __repr__(self) -> str:
        return f"Account(Username: '{self.username}',Password: '{self.password}', Account ID: '{self.acc_id}')"
    
    # Task Related Methods

    def view_tasks(self):
        """
        1. Display all tasks for the account.
        2. If no tasks exist, inform the user.
        3. If tasks exist, display each task's details.
        """
        
        if not self.tasks:
            print("No tasks available for this account.")
            return
        
        print(f"Tasks for Account '{self.username}':")
        for task_id, task in self.tasks.items():
            print(f"Task ID: {task_id}, Name: {task.task_name}, Status: {task.status}, Date Created: {task.date_created.strftime('%Y-%m-%d %H:%M:%S')}")

        
        
    def create_task(self):
        print("Creating a new task:" )

        # a dictionary for task status options
        status = {"1": "Not Started",
            "2": "In Progress",
            "3": "Completed",
            "4": "On Hold"
        }

        # Asking the task detials

        task_name: str = input("Enter the task name: ")
        description: str = input("Enter the task description: ")
        status_choice: str = input("""Enter the task status: 
                                (1: Not Started 
                                2: In Progress
                                3: Completed
                                4: On Hold)
                                : """)
        
        # Validate the status choice and set the task status
        if status_choice in status:
            task_status = status[status_choice]
        else:
            print("Invalid status choice. Defaulting to 'Not Started'.")
            task_status = status["1"]

        #Instantiate a new Task object

        new_task = Task(task_name, description, task_status, self.acc_id)
        
        self.tasks[new_task.task_id] = new_task

        print(f"""Task '{new_task.task_name}' created successfully with status '{new_task.status}'!
            Task_ID: {new_task.task_id}
            Description: {new_task.description}
            Date Created: {new_task.date_created.strftime('%Y-%m-%d %H:%M:%S')}
            Week Day Created: {new_task.date_created.strftime("%A")}""")
            
        new_task.day_created = new_task.date_created.strftime("%A")
        
        return new_task

    

    
    #Class Methods

    @classmethod
    def create_account(cls):

        """
    1. Loop until unique username + matching passwords
    2. Instantiate new account(...)
    3. Confirm creation, return to login flow
    """

        while True:
            
            print("Creating Account:")

            create_user: str  = input("Enter the Username: ")
    
            # Check if the username already exists
            if create_user in Account.DETAILS_ACCOUNT:
                print("Username already exists. Please try a different username.")
                continue
            
            first_password: str  = input("Enter the Password: ")
            final_password: str  = input("Confirm the Password: ")
            
            if first_password != final_password:
                print("Passwords do not match. Please try again.")
                continue
    
            print(f"Account created successfully for {create_user}!")
            return cls(create_user, final_password)
            break


#FUNCTION SECTIONS


def log_in():

    """
    1. Prompt for username/password
    2. Check against account.DETAILS_ACCOUNT
    3. On success: greet & return True
    4. On failure: call create_account() or exit
    """
    
    print("=======================================================================================================================================================")
    print("Log In:")
    log_username: str = input("\nEnter your username: ").lower()
    log_password: str = input("Enter your password: ")
    
    

    if log_username.lower() in Account.DETAILS_ACCOUNT and Account.DETAILS_ACCOUNT[log_username.lower()] == log_password:
        print(f"\nWelcome back, {log_username.title()}!")
        for acc in Account.ALL_ACC:
            if acc.username == log_username:
                global logged_acc
                logged_acc = acc
                break

        return logged_acc

    else:
        print("User is not found.")
        confirm: str  = input("\nDo you want to create an account? (y/n): ").lower()
        if confirm == "y":
            Account.create_account()
        else:
            print("Exiting the program. Please run again with the correct username.")
            exit()

In [ ]:
#FROM tasks.py

from datetime import datetime

#CLASS SECTION

class Task:

    statuses: list = ["Not Started", "In Progress", "Completed", "On Hold"]

    #Task Class Attributes
    ALL_TASKS: list = []
    NO_TASKS: int = 0
    LIST_TASK_ID: list[int] = []

    def __init__(self, task_name: str, description: str, status: str = "Not Started", owner_id: int = None):

        # Owner ID 
        self.owner_id = owner_id

        #Task ID generation
        next_id: int = Task.LIST_TASK_ID[-1] + 1 if Task.LIST_TASK_ID else 1
        self.task_id: int = next_id

        # Task details attributes
        
        self.task_name = task_name
        self.description = description
        self.status = status
        self.details = {}

        #Task date attributes
        self.date_created = datetime.now()
        self.due_date = None
        self.day_created = None
        

        # Add the new task to the list of all tasks and update the task count
        Task.ALL_TASKS.append(self)
        Task.NO_TASKS += 1
        Task.LIST_TASK_ID.append(self.task_id)
        
    def __repr__(self) -> str:
        return f"Task(Task ID: '{self.task_id}', Owner ID: '{self.owner_id}', Task Name: '{self.task_name}', Description: '{self.description}', Status: {self.status})"
    
    def __str__(self) -> str:
        return f"""
        ===========================
        Task Information
        ===========================
        Task ID: {self.task_id}
        Owner ID: {self.owner_id}

        ===========================
        Task Details:
        ===========================
        Task Name: {self.task_name}
        Description: {self.description}
        Status: {self.status}
        Date Created: {self.date_created}
        Due Date: {self.due_date}
        Day Created: {self.day_created}
        """


    def set_due_date(self):
            
            """
            1. Set the due date for the task.
            2. The user will be prompted to enter the year,
               month, and day for the due date.
            3. The month will be selected from a list of months,
               and the day must be valid for the selected month.
            4. The due date will be stored as a datetime object.
            5. If the user enters an invalid date, they will be prompted to try again
            6. The due date will be displayed to the user.
            """

            # Dictionary to map month numbers to month names 
            month_list = {"1": "January", "2": "February", "3": "March", "4": "April", "5":"May", "6": "June",
                            "7": "July", "8": "August", "9": "September", "10": "October", "11": "November", "12": "December"}
            
            # Dictionary to hold the maximum days in each month
            month_day_limit = {"January": 31, "February": 28, "March": 31, "April": 30, "May": 31, "June": 30,
                            "July": 31, "August": 31, "September": 30, "October": 31, "November": 30, "December": 31}
            
            print(f"Set the Due date of the task {self.task_name}: ")
            year = input("Enter the Year(eg. 2025): ").strip()

            while True:

                month_choice = input("""Enter the Month:
                            January - 1
                            February - 2
                            March   - 3
                            April   - 4
                            May     - 5
                            June    - 6
                            July    - 7
                            August  - 8
                            September - 9
                            October   - 10
                            November  - 11
                            December  - 12
                            : """).strip()
                
                
                
                if month_choice not in month_list:
                    print("Invalid Month Choice, Please try again. ")
                    continue
                
                month = month_list[month_choice]
                day_limit = month_day_limit[month]
                break
                    
            
            #Ensure the day not exceed the 31 day month limit
            while True:

                day = input("Enter the Day(eg. 1, 10, 25): ").strip()

                if int(day) > day_limit:
                    print("Invalid day, Please Enter a day within 31")
                    continue

                break
            
            self.due_date = datetime(int(year) if year.is_digit() else "Invalid Year Format", int(month_choice), day)
            print(f"{self.task_name} due date is now set!: {self.due_date}")

            return self.due_date
    




    def rename(self, new_task_name: str):
        """
        Change the name of the task.
        """

        if not new_task_name:
            print("Task name cannot be empty. Please provide a valid name.")
            return None

        confirmation = input(f"Are you sure you want to change the task name from {self.task_name} to {new_task_name}? (yes/no): ").strip().lower()

        if confirmation == "yes":

            self.task_name = new_task_name
            print(f"Task name changed to: {self.task_name}")
            return self.task_name
        
        else:
            print("Change Task Name Cancelled")
            return None
    



    
    def change_description(self, new_description: str):
        """
        Change the description of the task.
        """

        if not new_description:
            print("Description cannot be empty. Please provide a valid description.")
            return None
        
        confirmation = input(f"Are you sure you want to change the description of {self.task_name} to '{new_description}'? (yes/no): ").strip().lower()

        if confirmation == "yes":

            self.description = new_description
            print(f"Task description changed to: {self.description}")
            return self.description
        
        else:
            print("Change Description Cancelled")

            return None




    def change_status(self, new_status: str):

        if new_status not in Task.statuses:
            print(f"Invalid status. Please choose from: {', '.join(Task.statuses)}")
            return None
        
        confirmation = input(f"Are you sure you want to change the status of {self.task_name} to {new_status}? (yes/no): ").strip().lower()
        
        if confirmation == 'yes':
            self.status = new_status
            print(f"Status of {self.task_name} changed to: {self.status}")
            return self.status
        else:
            print("Status change cancelled.")
            return None

In [ ]:
#FROM menu.py

def super_menu(logged_acc):

    print(f"Welcome {logged_acc.username} to the To-Do Task Manager!")

    while True:
    
        print("\nPlease choose an option:")
        print("1. View Account Details")
        print("2. View Tasks")
        print("3. Create a New Task")
        print("4. Edit a Task")
        print("5. Log Out")

        choice = input("Enter your choice (1-5): ").strip()

        if choice == "1":
            print(logged_acc.full_acc_details())
        elif choice == "2":
            print(logged_acc.view_tasks())
        elif choice == "3":
            logged_acc.create_task()
        elif choice == "4":
            print(logged_acc.view_tasks())

            edit_id = input("\nEnter the Task ID you want to edit:").strip()
            if edit_id in logged_acc.tasks:
                logged_acc.edit_task(logged_acc, edit_id)
            else:
                print("Task ID not found.")
        
        elif choice == "5":
            print("Logging out...")
            break


def edit_task(logged_acc, task_id):
    """
    Edit an existing task.
    """

    task = logged_acc.tasks[task_id]


    print(f"Editing Task: {task.task_name}")

    print("Current Task Details:")
    print(f"Name: {task.task_name}")
    print(f"Description: {task.description}")
    print(f"Status: {task.status}")
    print(f"Due Date: {task.due_date.strftime('%Y-%m-%d') if task.due_date else 'No due date set'}")

    
    print("Please choose an option to edit:")
    print("1. Change Task Name")
    print("2. Change Task Description")
    print("3. Change Task Status")

    edit_choice = input("Enter your choice (1-3): ").strip()

    if edit_choice == "1":
        new_name = input("Enter the new task name: ").strip()
        task.rename(new_name)

    elif edit_choice == "2":
        new_description = input("Enter the new task description: ").strip()
        task.change_description(new_description)

    elif edit_choice == "3":
        new_status = input(f"Enter the new status for the task (current: {task.status}): ").strip()
        task.change_status(new_status)

    else:
        print("Invalid choice. Please try again.")

## Progress Log Update — July 6, 2025

---

## Final Project Reflection & Conclusion

**Date:** July 6, 2025
**Author:** Jerick C. Jualo

---

This marks the **final update** for my To‑Do List Manager project. Through building this application, I’ve demonstrated that I can leverage my Python knowledge, syntax proficiency, and problem‑solving skills (honed on CodeWars) to craft a real‑world tool. Highlights of the fundamentals applied:

* **Comments & Documentation**
  Clear docstrings and inline comments to explain intent and usage.

* **Variables & Operators**
  Thoughtful naming and use of arithmetic, comparison, and logical operators.

* **Core Data Structures**
  Lists, dictionaries, and class‐level registries to store and manage tasks and accounts.

* **Conditional Statements & Flow Control**
  `if`/`elif`/`else` logic for menu routing, input validation, and error handling.

* **Loops**
  While‑loops for login and menu loops; for‑loops for listing tasks and rebuilding registries.

* **Functions & User Input**
  Modular functions (e.g. `create_task()`, `edit_task()`) with prompts to gather and validate user input.

* **Data Type Manipulation**
  Converting between strings, integers, and `datetime` objects for statuses, IDs, and due dates.

* **Object‑Oriented Basics**

  * **Classes & Instances:** `Account` and `Task` encapsulate state and behavior.
  * **Encapsulation:** Methods validate and confirm changes before mutating object state.
  * **Inheritance & Polymorphism (Future Extension):** Foundation laid for subclassing or shared interfaces.

---

**Takeaway:**
This project solidifies my command of Python fundamentals and OOP principles. It’s a concrete example of translating coding challenge exercises into a fully modular, persistent application—bridging the gap between algorithmic practice and software design.


### 1. `main_module.py`

**Summary**

* Clear entry point (`main()`), guaranteed persistence, and modular orchestration of login, menu, and save workflows.

**Detailed Update**

* **Clear Entrypoint (`main()`)**

  * Introduced a dedicated `main()` function, encapsulating the full application workflow. This replaces ad‑hoc top‑level code with a single, testable entry point.
  * Added a comprehensive docstring outlining:

    1. Loading persisted data
    2. Authentication loop
    3. Interactive menu launch
    4. Guaranteed data save on exit

* **Persistence Integration**

  * Moved `load_data()` to the very start of `main()`, ensuring accounts and tasks are reconstructed before any user interaction.
  * Wrapped the main workflow in a `try…finally` block, calling `save_data()` in the `finally` clause to persist changes even on exceptions or unexpected logout.

* **Authentication Loop**

  * Initialized `logged_acc` as `None`, then looped until `log_in()` returned a valid `Account` instance.
  * Kept authentication logic in `classes_functs.log_in()`, letting `main()` focus on orchestration.

* **Menu Hand‑Off**

  * After successful login, `main()` passes the authenticated account to `super_menu(logged_acc)`, delegating all UI interactions to `menu.py`.
  * On exit from `super_menu()`, control returns to the `finally` block to trigger `save_data()`.

* **`if __name__ == "__main__"` Guard**

  * Wrapped the call to `main()` in the standard guard to prevent automatic execution on import, enabling safe unit testing and REPL use.

* **Top‑Level Banner & Introduction**

  * Consolidated program title and author credit into a triple‑quoted banner print.
  * Used consistent separator lines (e.g., `===…`) to break console sections and improve readability.

---

### 2. `menu.py`

**Summary**

* A centralized super‑menu loop for all post‑login user actions, with a dedicated edit helper and consistent UI patterns.

**Detailed Update**

* **Centralized Interaction Loop (`super_menu`)**

  * Encapsulates all post‑login choices in one function, taking an `Account` instance to ensure actions apply to the correct user.

* **Five Core Options**

  1. **View Account Details:** Displays username, ID, and task summary via the account’s own method.
  2. **View Tasks:** Lists each task’s ID, name, status, creation date, and due date in a tabular console format.
  3. **Create a New Task:** Delegates to `Account.create_task()`, keeping creation details inside the class.
  4. **Edit a Task:**

     * Re‑prints current tasks for reference.
     * Prompts for `task_id` and validates existence.
     * Hands off to `edit_task()` for modification workflows.
  5. **Log Out:** Exits the loop and returns control (and persistence) to `main()`.

* **Dedicated Edit Helper (`edit_task`)**

  * Separates the multi‑step editing dialogue from the main menu.
  * Displays the selected task’s current details up front.
  * Offers granular sub‑options: rename, change description, change status, change due date, delete.
  * Calls appropriate `Task` or `Account` methods per action.

* **Consistent Prompt/UI Patterns**

  * Uses separator lines to break up sections.
  * Standardizes input prompts (e.g., “Enter your choice (1–5):”).
  * Validates inputs and prompts “please try again” on invalid entries.

* **Separation of Concerns**

  * `menu.py` handles only UI routing and printing; no persistence or data logic.
  * Business logic resides in `Account` and `Task` classes.

* **Extensibility & Maintenance**

  * Adding new menu options or edit actions requires only adding a branch in `super_menu` or `edit_task`.
  * Menu functions accept object instances for easy unit testing with mocks.

---

### 3. `tasks.py`

**Summary**

* A full‑featured `Task` class with registries, auto‑increment IDs, rich representations, interactive setters, and serialization.

**Detailed Update**

* **Class‑Level Registries**

  * `ALL_TASKS`: holds every `Task` instance.
  * `LIST_TASK_ID`: tracks assigned task IDs for uniqueness.
  * `NO_TASKS`: running count of total tasks.

* **Auto‑Incrementing Task IDs**

  * In `__init__`, computes `task_id` as the next integer (last ID + 1 or 1 if none exist).
  * Appends the new ID to `LIST_TASK_ID` and increments `NO_TASKS`.

* **Owner Linkage**

  * Added an `owner_id` parameter in the constructor to associate each task with its `Account`.
  * Facilitates reattachment of tasks to accounts on load or creation.

* **Rich Representations**

  * `__repr__`: concise, developer‑friendly, single‑line summary.
  * `__str__`: multi‑line, human‑readable display of all task details.

* **Interactive Setters**

  * Consolidated date prompts and validation in `set_due_date()`, then wrapped in `change_due_date()` with confirmation.

* **Field‑Specific Updaters**

  * `rename()`, `change_description()`, `change_status()`: prompt for new values, validate, confirm, and update.

* **Scheduling Helpers**

  * Prepared structure for future methods like `days_until_due()`, `is_overdue()`, or `postpone()`.

* **Serialization Methods**

  * `to_dict()`: exports all attributes as JSON‑compatible primitives.
  * `from_dict()`: reconstructs a `Task`, restores registries, and re‑registers the instance.

* **Global Registry Maintenance**

  * Both `__init__` and `from_dict()` ensure tasks end up in `ALL_TASKS`, `LIST_TASK_ID`, and contribute to `NO_TASKS`.

---

### 4. `classes_functs.py` (`Account`)

**Summary**

* Renamed to `Account`, added docstrings, registries, automatic IDs, task management methods, and serialization.

**Detailed Update**

* **Capitalized Class Name & Docstrings**

  * Renamed `account` to `Account` to follow conventions, added a class‑level docstring.

* **Class‑Level Registries**

  * `LIST_ACC_ID`: tracks used account IDs.
  * `ALL_ACC`: holds every `Account` instance in memory.
  * `DETAILS_ACCOUNT`: maps usernames to passwords.
  * `NO_ACCOUNTS`: count of total accounts.

* **Automatic ID Generation**

  * In `__init__`, assigns a unique `acc_id` based on the last entry in `LIST_ACC_ID` or `1` if empty.
  * Appends the new ID and instance to `LIST_ACC_ID` and `ALL_ACC`, respectively.

* **Instance Registry & Task Namespace**

  * Initializes `self.tasks` as an empty dict for that user’s tasks.

* **Enhanced `__repr__`**

  * One‑line developer summary (username, password, ID).

* **Robust `create_account()` Static Method**

  * Prompts for and validates a unique username and matching password confirmation.
  * Returns the new `Account` instance immediately.

* **Authentication in `log_in()`**

  * Validates credentials against `DETAILS_ACCOUNT` (case‑insensitive lookup).
  * Returns the matching `Account` instance on success, or offers account creation on failure.

* **Account Display Methods**

  * `display_NO_ACCOUNTS()`, `display_accounts()`, and `view_acc_id()` for summaries and overviews.

* **Serialization Support**

  * `to_dict()`: exports `acc_id`, username, password, and task IDs.
  * `from_dict()`: reinstates an `Account` from its dict form, updating all registries.

---

### 5. `persistence.py`

**Summary**

* Centralized, atomic JSON I/O with robust load logic and strict separation of concerns.

**Detailed Update**

* **Atomic JSON Save**

  * `save_data()`: writes to `data.json.tmp` then renames to `data.json` to prevent corruption.
  * Serializes both accounts and tasks under top‑level keys `"accounts"` and `"tasks"`.

* **Unified Serialization Calls**

  * Uses each model’s `to_dict()` to build the JSON payload, automatically including future fields.

* **Robust Load Logic**

  * Checks for file existence and non‑empty content.
  * Catches `JSONDecodeError` and warns, allowing a fresh start instead of crashing.

* **Reconstruction Order**

  1. Rebuild accounts via `Account.from_dict()`.
  2. Rebuild tasks via `Task.from_dict()`.
  3. Reattach tasks to their owning accounts by matching `owner_id` to `acc_id`.

* **Separation of Concerns**

  * `persistence.py` deals solely with JSON I/O and calling class serializers; no UI or business logic.

* **Fail‑Safe Defaults**

  * Missing or empty files start the app with an empty state.
  * Corrupted files trigger a console warning but do not halt execution.

---

## Overall Impact

Every component is now self‑documenting, testable, and resilient—providing a robust, modular, and persistent To‑Do List Manager.


In [ ]:
#FROM main_module.py

from persistence import load_data, save_data
from classes_functs import log_in, Account
from menu import super_menu

# Introduction

print("""welcome to my To-Do List Manager
      
Coded by: Jerick C. Jualo""")

print("\n=======================================================================================================================================================")
print("Welcome to the To-Do Task Manager!")
print("=======================================================================================================================================================")
print("This program will help you track your goals and weekly schedule for Python, SQL Learning, and Math Study.")
print("Also, it will help you create your tasks for today and check the status of your tasks.")
print("Please log in to continue.")
print("=======================================================================================================================================================\n")


# Ensure the user is logged in before proceeding 


def main():
    """
    Entry point for the To‑Do Task Manager application.

    Workflow:
      1. load_data(): Rebuild in-memory Account and Task objects from disk.
      2. Prompt the user to log in or create an account.
      3. Once authenticated, enter the interactive super_menu loop.
      4. On exit (user logs out or an unexpected error occurs), save_data() is
         always called to persist the current state to disk.

    This ensures that no matter how the program terminates, all changes
    made during the session are written back to 'data.json'.
    """
    
    load_data()
    try:
        print("Welcome to To‑Do Task Manager…")
        logged_acc = None

        # Keep prompting until a valid Account is returned
        while not logged_acc:
            logged_acc = log_in()

        # Launch the main interactive menu for the authenticated user
        super_menu(logged_acc)
    finally:
        # Always persist data even on unexpected exit
        save_data()

if __name__ == "__main__":
    # Only run the application loop if this script is executed directly
    main()



print("\n=======================================================================================================================================================")
print(f"Here's Jerick's Goals for this Summer Vacation")
print("=======================================================================================================================================================")
print("Goals:")
print("1. Solidify Python skills from Basic to Data Collections to Functions to File and Error Handling to OOP")
print("2. Learn PostgreSQL")
print("3. Study Math/Statisticxs for Data Science At least Study and Solidify my Math Skills in Algebra")
print("=======================================================================================================================================================")

weekly_schedule = input("\nDo you want to see the Weekly Schedule for Python, SQL Learning and Math Study? (yes/no): ").strip().lower()

if weekly_schedule == "yes":
       print("\nHere's the Weekly Schedule for Python, SQL Learning and Math Study")
       print("=======================================================================================================================================================")
       print("""
       Sunday - Do the Toy Projects in Python
              - Rest and Relax
       
       Monday - Study/Code in Python or Do Problem Solving in Python Problems provided by ChatGPT or HackerRank, LeetCode, Codewars
              - Study Math(Getting Ready for Algebra Course in Khan Academy)
       
       Tuesday - Study/Code in Python or Do Problem Solving in Python Problems provided by ChatGPT or HackerRank, LeetCode, Codewars
               - Study SQL/PostgreSQL and Do Problem Solving in SQL Problems provided by ChatGPT

       Wednesday - Study/Code in Python or Do Problem Solving in Python Problems provided by ChatGPT or HackerRank, LeetCode, Codewars
                 - Study Math(Getting Ready for Algebra Course in Khan Academy)
      
       Thursday - Study/Code in Python or Do Problem Solving in Python Problems provided by ChatGPT or HackerRank, LeetCode, Codewars
                - Study SQL/PostgreSQL and Do Problem Solving in SQL Problems provided by ChatGPT

       Friday - Study/Code in Python or Do Problem Solving in Python Problems provided by ChatGPT or HackerRank, LeetCode, Codewars
              - Study Math(Getting Ready for Algebra Course in Khan Academy)

       Saturday - Do the Toy Projects in Python 
                - Study SQL/PostgreSQL and Do Problem Solving in SQL Problems provided by ChatGPT\n""")
       print("=======================================================================================================================================================\n")
         
 # i change the rotation of tasks from monday to saturday and only sunday is the toy project in python
 # as i now relized that i need to rest and only do the toy projects in python on sunday



In [ ]:
#FROM menu.py

def super_menu(logged_acc):

    """
    Display the main interactive menu for a logged-in Account and route user choices
    to the appropriate account/task operations.

    This menu loops until the user chooses to log out. It provides options to:
      1. View account details
      2. View all tasks for the account
      3. Create a new task under the account
      4. Edit an existing task
      5. Log out and exit the menu loop

    Args:
        logged_acc (Account): The currently authenticated Account instance whose
                              data and tasks will be manipulated.

    Side Effects:
        - Prints menu prompts and separators to standard output.
        - Calls Account methods (full_acc_details, view_tasks, create_task).
        - Calls the edit_task() helper to handle task modifications.
    """

    print("\n=======================================================================================================================================================")
    print(f"Welcome back {logged_acc.username} to the To-Do Task Manager!")

    while True:
        print("\n=======================================================================================================================================================")
        print("\nPlease choose an option:")
        print("1. View Account Details")
        print("2. View Tasks")
        print("3. Create a New Task")
        print("4. Edit a Task")
        print("5. Log Out")
        print("========================================================================================================================================================")

        choice = input("Enter your choice (1-5): ").strip()

        if choice == "1":
            # Show full account info including all tasks
            print(logged_acc.full_acc_details())
        elif choice == "2":
            # List summary view of tasks
            print(logged_acc.view_tasks())
        elif choice == "3":
            # Prompt and create a new task under this account
            logged_acc.create_task()
        elif choice == "4":
            # Show tasks and then prompt for which task to edit
            print(logged_acc.view_tasks())

            edit_id = input("\nEnter the Task ID you want to edit:").strip()

            if int(edit_id) in logged_acc.tasks:
                edit_task(logged_acc, int(edit_id))
            else:
                print("Task ID not found.")
        
        elif choice == "5":
            # Exit the menu loop
            print("Logging out...")
            break


def edit_task(logged_acc, task_id):
    """
    Provide an interactive sub-menu for editing or deleting a single Task.

    Displays current task details (name, description, status, due date),
    then offers options to:
      1. Change the task's name
      2. Change the task's description
      3. Change the task's status
      4. Change the task's due date
      5. Delete the task entirely

    After the user selects an option, the corresponding Task or Account
    method is invoked to perform the change.

    Args:
        logged_acc (Account): The Account instance that owns the task.
        task_id (int):       The unique identifier of the Task to edit.

    Side Effects:
        - Prints detailed task information and edit prompts.
        - Calls Task.rename(), change_description(), change_status(),
          change_due_date(), or Account.delete_task() based on user input.
    """

    task = logged_acc.tasks[task_id]

    print("\n=======================================================================================================================================================")
    print(f"Editing Task: {task.task_name}")

    print("Current Task Details:")
    print(f"Name: {task.task_name}")
    print(f"Description: {task.description}")
    print(f"Status: {task.status}")
    print(f"Due Date: {task.due_date.strftime('%Y-%m-%d') if task.due_date else 'No due date set'}")

    print("=======================================================================================================================================================\n")

    
    print("Please choose an option to edit:")
    print("1. Change Task Name")
    print("2. Change Task Description")
    print("3. Change Task Status")
    print("4. Change Task Due Date")
    print("5. Delete Task")

    print("========================================================================================================================================================")

    edit_choice = input("Enter your choice (1-5): ").strip()

    if edit_choice == "1":

        print("=========================================================================================================================================================")
        new_name = input("Enter the new task name: ").strip()
        task.rename(new_name)

    elif edit_choice == "2":

        print("=========================================================================================================================================================")
        new_description = input("Enter the new task description: ").strip()
        task.change_description(new_description)

    elif edit_choice == "3":

        print("=========================================================================================================================================================")
        print("Please choose from: Not Started, In Progress, Completed, On Hold")
        new_status = input(f"Enter the new status for the task (current: {task.status}): ").strip()
        task.change_status(new_status)

    elif edit_choice == "4":
        # Delegate to the Task’s due-date setter

        task.change_due_date()

    elif edit_choice == "5":
        # Remove the task from both the Account and global registries
        logged_acc.delete_task(task_id)
        
    else:
        print("Invalid choice. Please try again.")

In [ ]:
#FROM classes_functs.py

from datetime import datetime 
from tasks import Task

#CLASS FOR ACCOUNT
class Account:

    """
    Represents a user account in the To-Do Task Manager, handling authentication
    and providing a namespace for that user's tasks.

    Attributes:
        username (str): The account's login name.
        password (str): The account's password (plaintext; consider hashing for real apps).
        acc_id (int): Unique identifier for this account.
        tasks (dict[int, Task]): Tasks owned by this account, keyed by task_id.

    Class Attributes:
        LIST_ACC_ID (list[int]): All assigned account IDs.
        ALL_ACC (list[Account]): All Account instances created.
        NO_ACCOUNTS (int): Total number of accounts.
        DETAILS_ACCOUNT (dict[str, str]): Mapping of lowercase username -> password for login.
    """
    
    # Class Attributes
    LIST_ACC_ID: list[int] = []
    ALL_ACC: list[str] = []
    NO_ACCOUNTS: int = 0
    DETAILS_ACCOUNT = {}


        
    
    def __init__(self, username: str, password: str):
        """
        Initialize a new Account, assign a unique acc_id, and register in class-level stores.

        Args:
            username (str): Desired username (must be unique).
            password (str): Password for the account.
        """

        # Username, Password, Account ID
        key = username.lower()
        self.username = username
        self.password = password
        
        next_id: int = Account.LIST_ACC_ID[-1] + 1 if Account.LIST_ACC_ID else 1

        self.acc_id: int = next_id # Owner ID on Task Class
        
        #Task related
        
        self.tasks = {}
        

        #Actions to be executed for Class Attributes
        Account.LIST_ACC_ID.append(next_id)
        Account.ALL_ACC.append(self)
        Account.NO_ACCOUNTS += 1
        Account.DETAILS_ACCOUNT[key] = password 
        


    # Instance Methods

    #Account Details Display Methods

    def display_NO_ACCOUNTS(self) -> str:
        """
        Return a string showing the total number of accounts.

        Returns:
            str: e.g. "Total Number of Accounts: 5".
        """
        return f"Total Number of Accounts: {Account.NO_ACCOUNTS}"
    
    
    def display_accounts(self) -> str:
        """
        Return a summary of all registered accounts and their passwords.

        Returns:
            str: e.g. "Account Details: {'alice': 'pass123', 'bob': 'xYz'}".
        """
        return f"Account Details: {Account.DETAILS_ACCOUNT}"

    def view_acc_id(self) -> str:
        """
        Display this account's username and ID.

        Returns:
            str: e.g. "Account: alice ID: 1".
        """
        return f"Account: {self.username} ID: {self.acc_id}"
    
    
    def full_acc_details(self) -> str:
        """
        Print and return a detailed summary of this account and its tasks.

        Side Effects:
            Prints account info and each task's __repr__ to stdout.

        Returns:
            str: Empty string for chaining with print().
        """

        print("==========================")
        print(f"Account Details for {self.username}:")
        print(f"Username: {self.username}")
        print(f"Account ID: {self.acc_id}")
        print("========================================================================================================================================================")

        for x in self.tasks.values():
            print(x)
        print("========================================================================================================================================================")
        return ""

        
    def __repr__(self) -> str:
        """
        Developer-facing representation of the account.
        """
        return f"Account(Username: '{self.username}',Password: '{self.password}', Account ID: '{self.acc_id}')"
    


    # Task Related Methods

    def view_tasks(self):
        """
        1. Display all tasks for the account.
        2. If no tasks exist, inform the user.
        3. If tasks exist, display each task's details.

        Side Effects:
            Prints each task's ID, name, status, date created, and due date.
        """
        
        if not self.tasks:
            print("No tasks available for this account.")
            return
        print("========================================================================================================================================================")
        print(f"Tasks for Account '{self.username}':")
        print("========================================================================================================================================================")
        for task_id, task in self.tasks.items():
            print(f"Task ID: {task_id}, Name: {task.task_name}, Status: {task.status}, Date Created: {task.date_created.strftime('%Y-%m-%d %H:%M:%S')} , Due Date: {task.due_date.strftime('%Y-%m-%d') if task.due_date else 'No due date set'}")

        print("========================================================================================================================================================")



    def create_task(self):
        """
        Interactively create a new Task, register it with this account, and return it.

        Returns:
            Task: The newly created Task instance.
        """

        print("\n========================================================================================================================================================")
        print("Creating a new task:\n" )

        # a dictionary for task status options
        status = {"1": "Not Started",
            "2": "In Progress",
            "3": "Completed",
            "4": "On Hold"
        }

        # Asking the task detials

        task_name: str = input("Enter the task name: ")
        description: str = input("Enter the task description: ")
        status_choice: str = input("""Enter the task status: 
                                (1: Not Started 
                                2: In Progress
                                3: Completed
                                4: On Hold)
                                : """)
        
        # Validate the status choice and set the task status
        if status_choice in status:
            print("========================================================================================================================================================")
            task_status = status[status_choice]
        else:
            print("Invalid status choice. Defaulting to 'Not Started'.")
            task_status = status["1"]

        #Instantiate a new Task object

        new_task = Task(task_name, description, task_status, self.acc_id)
        
        self.tasks[new_task.task_id] = new_task

        print(f"""Task '{new_task.task_name}' created successfully with status '{new_task.status}'!
            Task_ID: {new_task.task_id}
            Description: {new_task.description}
            Date Created: {new_task.date_created.strftime('%Y-%m-%d %H:%M:%S')}
            Week Day Created: {new_task.date_created.strftime("%A")}""")
            
        new_task.day_created = new_task.date_created.strftime("%A")
        
        return new_task
    



    def delete_task(self, task_id: int):

        """
        Remove a Task by its ID after user confirmation.

        Args:
            task_id (int): Identifier of the task to delete.

        Returns:
            bool: True if deleted, False otherwise.

        Side Effects:
            Updates both self.tasks and global Task registries.
        """
        
        if task_id in self.tasks:
            confirm = input("Do you really want to delete this task? (yes/no): ").strip().lower()

            if confirm == "yes":
                task = self.tasks.pop(task_id)

                if task in Task.ALL_TASKS:
                    Task.ALL_TASKS.remove(task)
                if task_id in Task.LIST_TASK_ID:
                    Task.LIST_TASK_ID.remove(task_id)
                Task.NO_TASKS -= 1

                if Task.NO_TASKS > 0:
                    Task.NO_TASKS -= 1
                
                print(f"Task with ID {task_id} has been deleted.")

                return True
            
            else:
                print("Task deletion cancelled.")
        else:
            print(f"Task with ID {task_id} does not exist in this account.")

    
    #Class Methods

    @classmethod
    def create_account(cls):

        """
        Interactively prompt for a unique username and matching password to
        register a new Account.

        Returns:
            Account: The newly created account.
        """

        while True:
            
            print("Creating Account:")

            create_user: str  = input("Enter the Username: ")
    
            # Check if the username already exists
            if create_user in Account.DETAILS_ACCOUNT:
                print("Username already exists. Please try a different username.")
                continue
            
            first_password: str  = input("Enter the Password: ")
            final_password: str  = input("Confirm the Password: ")
            
            if first_password != final_password:
                print("Passwords do not match. Please try again.")
                continue
    
            print(f"Account created successfully for {create_user}!")
            return cls(create_user, final_password)
            break



    def to_dict(self) -> dict:
        """
        Serialize this Account's core data into a JSON-serializable dict.

        Returns:
            dict: Contains 'acc_id', 'username', 'password', and 'task_ids'.
        """

        return {
            "acc_id":   self.acc_id,
            "username": self.username,
            "password": self.password,
            "task_ids": list(self.tasks.keys()),
        }

    @classmethod
    def from_dict(cls, data: dict):
        """
        Reconstruct an Account from its serialized dict, updating class registries.

        Args:
            data (dict): Serialized account data.

        Returns:
            Account: The re-hydrated account instance.
        """
        acct = cls.__new__(cls)

        # 2) Manually set instance attributes
        acct.username = data["username"]
        acct.password = data["password"]
        acct.acc_id   = data["acc_id"]
        acct.tasks    = {}           

        
        Account.LIST_ACC_ID.append(acct.acc_id)
        Account.ALL_ACC.append(acct)
        Account.NO_ACCOUNTS += 1
        Account.DETAILS_ACCOUNT[acct.username.lower()] = acct.password

        return acct


#FUNCTION SECTIONS


def log_in():

    """
    Prompt the user for login credentials, authenticate, and return the Account.

    Returns:
        Account: Logged-in account instance.

    Side Effects:
        - Calls create_account() on failure if user opts in.
        - Exits program on cancellation.
    """
    
    print("=======================================================================================================================================================")
    print("Log In:")
    log_username: str = input("\nEnter your username: ").lower()
    log_password: str = input("Enter your password: ")
    print("=======================================================================================================================================================")
    
    

    if log_username.lower() in Account.DETAILS_ACCOUNT and Account.DETAILS_ACCOUNT[log_username.lower()] == log_password:
        for acc in Account.ALL_ACC:
            if acc.username == log_username:
                global logged_acc
                logged_acc = acc
                break

        return logged_acc

    else:
        print("User is not found.")
        confirm: str  = input("\nDo you want to create an account? (y/n): ").lower()
        if confirm == "y":
            Account.create_account()
        else:
            print("Exiting the program. Please run again with the correct username.")
            exit()









In [ ]:
#FROM tasks.py

from datetime import datetime

#CLASS SECTION

class Task:

    """
    Represents a single to-do task with an owner, description, status, and scheduling attributes.
    Tracks all tasks globally and supports operations for updating and serializing.
    """

    # Allowed statuses for any task
    statuses: list = ["Not Started", "In Progress", "Completed", "On Hold"]

    #Task Class Attributes
    ALL_TASKS: list = []
    NO_TASKS: int = 0
    LIST_TASK_ID: list[int] = []

    def __init__(self, task_name: str, description: str, status: str = "Not Started", owner_id: int = None):

        """
        Initialize a new Task instance.

        Args:
            task_name (str): Short title of the task.
            description (str): Detailed description of the task.
            status (str): Current status; must be one of Task.statuses.
            owner_id (int): Identifier of the owning Account.
        """

        # Owner ID 
        self.owner_id = owner_id

        #Task ID generation
        next_id: int = Task.LIST_TASK_ID[-1] + 1 if Task.LIST_TASK_ID else 1
        self.task_id: int = next_id

        # Task details attributes
        
        self.task_name = task_name
        self.description = description
        self.status = status
        self.details = {}

        #Task date attributes
        self.date_created = datetime.now()
        self.due_date = None
        self.day_created = None
        

        # Add the new task to the list of all tasks and update the task count
        Task.ALL_TASKS.append(self)
        Task.NO_TASKS += 1
        Task.LIST_TASK_ID.append(self.task_id)

        
    def __repr__(self) -> str:

        """
        Return a concise developer-facing representation of the Task.
        """

        return f"Task(Task ID: '{self.task_id}', Owner ID: '{self.owner_id}', Task Name: '{self.task_name}', Description: '{self.description}', Status: {self.status})"
    
    def __str__(self) -> str:

        """
        Return a human-friendly, multi-line description of the Task, including
        scheduling and status information.
        """

        return f"""
        ===========================
        Task Information
        ===========================
        Task ID: {self.task_id}
        Owner ID: {self.owner_id}

        ===========================
        Task Details:
        ===========================
        Task Name: {self.task_name}
        Description: {self.description}
        Status: {self.status}
        Date Created: {self.date_created}
        Due Date: {self.due_date}
        Day Created: {self.day_created}
        """


    def set_due_date(self):
            
            """
            1. Set the due date for the task.
            2. The user will be prompted to enter the year,
               month, and day for the due date.
            3. The month will be selected from a list of months,
               and the day must be valid for the selected month.
            4. The due date will be stored as a datetime object.
            5. If the user enters an invalid date, they will be prompted to try again
            6. The due date will be displayed to the user.
            """

            # Dictionary to map month numbers to month names 
            month_list = {"1": "January", "2": "February", "3": "March", "4": "April", "5":"May", "6": "June",
                            "7": "July", "8": "August", "9": "September", "10": "October", "11": "November", "12": "December"}
            
            # Dictionary to hold the maximum days in each month
            month_day_limit = {"January": 31, "February": 28, "March": 31, "April": 30, "May": 31, "June": 30,
                            "July": 31, "August": 31, "September": 30, "October": 31, "November": 30, "December": 31}
            
            while True:

                print(f"Set the Due date of the task {self.task_name}:")

                year: str = input("Enter the Year(eg. 2025): ").strip()

                if len(year) != 4 or not year.isdigit() and year > self.date_created.strftime("%Y"):
                    print("Invalid Year Format, Please Enter a valid year in YYYY format.")
                    continue
                else:
                    break

            while True:

                month_choice: str = input("""Enter the Number of the Month:
                            January - 1
                            February - 2
                            March   - 3
                            April   - 4
                            May     - 5
                            June    - 6
                            July    - 7
                            August  - 8
                            September - 9
                            October   - 10
                            November  - 11
                            December  - 12
                            : """).strip()
                
                
                
                if month_choice not in month_list:
                    print("Invalid Month Choice, Please try again. ")
                    continue
                
                month = month_list[month_choice]
                day_limit = month_day_limit[month]
                break
                    
            
            #Ensure the day not exceed the 31 day month limit
            while True:

                day = input("Enter the Day(eg. 1, 10, 25): ").strip()

                if int(day) > day_limit:
                    print(f"Invalid day, Please Enter a day within {month_day_limit[month]} days limit for {month}.")
                    continue

                break

            y = int(year)
            m = int(month_choice)
            d = int(day)
            
            self.due_date = datetime(y, m, d)
            print(f"{self.task_name} due date is now set!: {self.due_date}")

            return self.due_date
    




    def rename(self, new_task_name: str):
        """
        Change the task's name after user confirmation.

        Args:
            new_task_name (str): The proposed new name for the task.

        Returns:
            str or None: The updated name if changed; None if cancelled or invalid.
        """

        if not new_task_name:
            print("Task name cannot be empty. Please provide a valid name.")
            return None

        confirmation = input(f"Are you sure you want to change the task name from {self.task_name} to {new_task_name}? (yes/no): ").strip().lower()

        if confirmation == "yes":

            self.task_name = new_task_name
            print(f"Task name changed to: {self.task_name}")
            return self.task_name
        
        else:
            print("Change Task Name Cancelled")
            return None
    



    
    def change_description(self, new_description: str):
        """
        Change the task's description after user confirmation.

        Args:
            new_description (str): The new description text.

        Returns:
            str or None: The updated description if changed; None if cancelled or invalid.
        """

        if not new_description:
            print("Description cannot be empty. Please provide a valid description.")
            return None
        
        confirmation = input(f"Are you sure you want to change the description of {self.task_name} to '{new_description}'? (yes/no): ").strip().lower()

        if confirmation == "yes":

            self.description = new_description
            print(f"Task description changed to: {self.description}")
            return self.description
        
        else:
            print("Change Description Cancelled")

            return None




    def change_status(self, new_status: str):

        """
        Update the task's status, ensuring it matches one of the allowed
        statuses, and confirm before applying.

        Args:
            new_status (str): The new status value.

        Returns:
            str or None: The updated status if changed; None if cancelled or invalid.
        """

        if new_status not in Task.statuses:
            print(f"Invalid status. Please choose from: {', '.join(Task.statuses)}")
            return None
        
        confirmation = input(f"Are you sure you want to change the status of {self.task_name} to {new_status}? (yes/no): ").strip().lower()
        
        if confirmation == 'yes':
            self.status = new_status
            print(f"Status of {self.task_name} changed to: {self.status}")
            return self.status
        else:
            print("Status change cancelled.")
            return None
        
    def change_due_date(self):
        """
        Wrapper around set_due_date() that asks for user confirmation to
        overwrite the existing due date.

        Returns:
            datetime or None: The updated due_date if changed; None if cancelled.
        """

        new_due_date = self.set_due_date()
        
        confirmation = input(f"Are you sure you want to change the due date of {self.task_name} to {self.due_date}? (yes/no): ").strip().lower()

        if confirmation == "yes":
            self.due_date = new_due_date
            print(f"Due date changed to: {self.due_date}")
            return self.due_date
        
        else:
            print("Change Due Date Cancelled")
            return None
        
    # METHODS FOR SERIALIZATION
    
    def to_dict(self) -> dict:
        """
        Serialize this Task to a JSON-serializable dictionary.

        Returns:
            dict: A mapping of task attributes to basic Python types.
        """

        return {
            "task_id":      self.task_id,
            "owner_id":     self.owner_id,
            "task_name":    self.task_name,
            "description":  self.description,
            "status":       self.status,
            "date_created": self.date_created.isoformat(),
            "due_date":     self.due_date.isoformat() if self.due_date else None,
            "day_created":  self.day_created,
        }

    @classmethod
    def from_dict(cls, data: dict):
        """
        Re-hydrate a Task instance from its dictionary form, restoring
        all attributes and updating class-level registries.

        Args:
            data (dict): The serialized task dictionary.

        Returns:
            Task: The reconstructed Task object.
        """
        
        task = cls(
            data["task_name"],
            data["description"],
            data["status"],
            data["owner_id"],
        )
       
        task.task_id      = data["task_id"]
        task.date_created = datetime.fromisoformat(data["date_created"])
        task.due_date     = datetime.fromisoformat(data["due_date"]) if data["due_date"] else None
        task.day_created  = data["day_created"]
        

        Task.LIST_TASK_ID.append(task.task_id)
        Task.ALL_TASKS.append(task)
        Task.NO_TASKS += 1
        return task

In [ ]:
#FROM persistence.py

"""
persistence.py

Handles saving and loading persistent data for the To-Do Task Manager.
This module ensures all user accounts and their tasks are written to and read from disk,
allowing the application to retain state across sessions.

Data is serialized to JSON and stored in 'data.json'. The structure includes:
- A list of serialized account dictionaries.
- A list of serialized task dictionaries.
"""

import json
from pathlib import Path
from classes_functs import Account
from tasks import Task

# Path to the JSON file where data is persisted
DATA_FILE = Path("data.json")

def save_data():
    """
    Save all accounts and tasks to a JSON file atomically.

    This function serializes all current instances of Account and Task
    into a structured JSON object and writes it to 'data.json'.
    A temporary file is written first and then renamed to avoid data loss
    in case of an interruption.

    Structure:
    {
        "accounts": [...],
        "tasks": [...]
    }"""


    payload = {
        "accounts": [acct.to_dict() for acct in Account.ALL_ACC],
        "tasks":    [task.to_dict() for task in Task.ALL_TASKS],
    }
    tmp = DATA_FILE.with_suffix(".tmp")
    with tmp.open("w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)
    tmp.replace(DATA_FILE)



def load_data():
    """
    Load all accounts and tasks from the JSON file (if it exists).

    This function attempts to read 'data.json' and deserialize its contents
    into Account and Task objects. If the file is missing or empty, nothing
    is loaded. If the file is malformed, a warning is printed and loading is skipped.

    Reconstructed accounts are added to the Account registries.
    Tasks are reconstructed and also attached to their respective owner accounts.
    """
    if not DATA_FILE.exists():
        return
    
    text = DATA_FILE.read_text(encoding="utf-8").strip()
    
    if not text:
        # empty file → treat as no data
        return

    try:
        payload = json.loads(text)
    except json.JSONDecodeError:
        print("Warning: data.json is corrupted or not valid JSON—starting fresh.")
        return


    with DATA_FILE.open("r", encoding="utf-8") as f:
        payload = json.load(f)

    # First recreate accounts
    for acct_data in payload.get("accounts", []):
        acct = Account.from_dict(acct_data)
        # Account.from_dict should append itself into ALL_ACC, LIST_ACC_ID, etc.

    # Then recreate tasks and attach to owners
    for task_data in payload.get("tasks", []):
        task = Task.from_dict(task_data)
        # Task.from_dict should append to ALL_TASKS, LIST_TASK_ID, NO_TASKS
        # Now also attach to account.tasks:
        owner = next((a for a in Account.ALL_ACC if a.acc_id == task.owner_id), None)
        if owner:
            owner.tasks[task.task_id] = task
